# Photometry Analysis


## import and definition


In [ ]:
import pandas as pd
import numpy as np

from routine.plotting import plot_signals, plot_peaks
from routine.processing import compute_dff, find_pks
from routine.utilities import load_data

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
IN_DATA = "./data/d1/data.csv"
IN_TS = "./data/d1/timestamps.csv"
IN_LOC = "./data/d1/xy_coordinates.csv"
IN_INJ = "./data/d1/injection_time.csv"
PARAM_ROIS = ["Region0R", "Region1R", "Region2G", "Region3G"]
PARAM_NFM_DISCARD = 300
PARAM_EVT_RANGE = (-200, 400)
PARAM_PK_THRES = 5
PARAM_PK_FREQ_WD = 100
PARAM_LED_DICT = {7: "initial", 1: "415nm", 2: "470nm", 4: "560nm"}


## load data


ensure equal number of frames for each channel


In [ ]:
data, ts = load_data(IN_DATA, IN_TS, PARAM_NFM_DISCARD, PARAM_LED_DICT)


## visualize raw signal


In [ ]:
plot_signals(data, PARAM_ROIS)


## compute df/F


In [ ]:
data_dff = compute_dff(data, PARAM_ROIS)


## visualize correction result


In [ ]:
plot_signals(data_dff, PARAM_ROIS, default_window=None)


## find peaks and visualize result


In [ ]:
data_pks = find_pks(
    data_dff, PARAM_ROIS, PARAM_PK_THRES, PARAM_PK_FREQ_WD, ["415nm-dff", "470nm-dff"]
)


In [ ]:
plot_peaks(data_pks, PARAM_ROIS)


## align with behavior


In [ ]:
loc_df = pd.read_csv(IN_LOC, names=["t(ms)", "x", "y"], skiprows=1)
inj_ts = pd.read_csv(IN_INJ, header=None).iloc[0, 0] - data_dff["Timestamp"].min()
loc_df["dist"] = np.sqrt(loc_df["x"] ** 2 + loc_df["y"] ** 2)
loc_df["t"] = (loc_df["t(ms)"] - loc_df["t(ms)"].min()) / 1000
loc_df["dt"] = loc_df["t"].diff()
loc_df["v"] = loc_df["dist"] / loc_df["dt"]


In [ ]:
dat_dff = data_dff[data_dff["signal"] == "415nm-dff"].copy()
dat_pks = data_pks[data_pks["signal"] == "415nm-dff"].copy()
dat_dff["t"] = dat_dff["Timestamp"] - dat_dff["Timestamp"].min()
dat_pks["t"] = dat_pks["Timestamp"] - dat_pks["Timestamp"].min()
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=dat_dff["t"], y=dat_dff["Region0R"]), row=1, col=1)
fig.add_trace(go.Scatter(x=dat_pks["t"], y=dat_pks["Region0R-freq"]), row=2, col=1)
fig.add_trace(go.Scatter(x=loc_df["t"], y=loc_df["v"]), row=3, col=1)
fig.update_xaxes(title_text="Time (sec)")
fig.update_yaxes(title_text="df/F 415nm", row=1, col=1)
fig.update_yaxes(title_text="spike frequency", row=2, col=1)
fig.update_yaxes(title_text="velocity", row=3, col=1)
fig.add_vline(inj_ts, line_dash="dot", line_color="grey")
